Narges Ghanbari

99442236

In [1]:
pip install binarytree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 1.4 MB/s eta 0:00:00


In [19]:
from random import random, randint, seed,choice
from statistics import mean
from copy import deepcopy
from binarytree import Node
from collections import deque

from math import sin as math_sin, cos as math_cos, radians, pow


POP_SIZE        = 60   # population size
MIN_DEPTH       = 2    # minimal initial random tree depth
MAX_DEPTH       = 5    # maximal initial random tree depth
GENERATIONS     = 250  # maximal number of generations to run evolution
TOURNAMENT_SIZE = 5    # size of tournament for tournament selection
XO_RATE         = 0.8  # crossover rate
PROB_MUTATION   = 0.2  # per-node mutation probability


def is_float(s):
    try:
        float(s)
        return True
    except ValueError:
        return False


add = lambda x, y: x + y
sub = lambda x, y: x - y
mul = lambda x, y: x * y
pow = lambda x,y: 0 if x==0 else pow(x,y)
div = lambda x,y: x/y if y!=0 else x
cos = lambda x: math_cos(radians(x))
sin = lambda x:math_sin(radians(x))





FUNCTIONS = ['add', 'sub', 'mul', 'div', 'sin', 'cos']
TERMINALS = ['x']

def target_func(x): # evolution's target
    return 3*x+5

def generate_dataset(target_func): # generate 101 data points from target_func
    dataset = []
    for x in range(-100,101,2):
        x /= 100
        dataset.append([x, target_func(x)])
    return dataset

class GPTree:
    def __init__(self,node=Node(1)):
        self.root=node
    def __str__(self):
      print(self.root)
      return ''

    @classmethod
    def from_string(cls,expression):
      if not expression:
        return
      s=deque()
      for token in (expression.split()):
        if token in FUNCTIONS:
          # pop two nodes `x` and `y` from the stack
            x = s.pop()
            y = s.pop()
            #construct a new binary tree whose root is the operator and whose
            #left and right children point to 'y' and 'x'
            node=Node(token, y, x)

            #push the current node into the stack
            s.append(node)
           # if the current token is an operand, create a new binary tree node
        # whose root is the operand and push it into the stack
        else:
            s.append(Node(token))
      return cls(s[-1])


    def compute(self,variables,node=None):
      #variables format: {'x':2,'y':5, ...}

      if node is None:
        node=self.root
      #if the node is a function , apply it to its children
      if node.value in FUNCTIONS:
        func=globals()[node.value]
        if node.value in ['sin','cos']:
          return func(self.compute(variables,node.left))
        else:
          return func(self.compute(variables,node.left),self.compute(variables,node.right))

      #if the node is a terminal, return its value
      elif is_float(node.value):
        return float(node.value)

      #if the node is a variable
      elif node.value in variables.keys():
        return variables[node.value]

    @classmethod
    def random_tree(cls,grow, max_depth, depth = 0):
        node=cls()
        if depth<MIN_DEPTH or (depth<max_depth and not grow):
          func=FUNCTIONS[randint(0, len(FUNCTIONS) - 1)]
          node.root=Node(func)

          if func in ['sin','cos']:
            node.root.left=cls().random_tree(grow, max_depth, depth=depth + 1).root
          else:
            node.root.left = cls().random_tree(grow, max_depth, depth=depth + 1).root
            node.root.right = cls().random_tree(grow, max_depth, depth=depth + 1).root

        #add termianl to leaf nodes
        elif depth>=max_depth:
          if random()<0.2:
            node.root=Node(TERMINALS[randint(0,len(TERMINALS)-1)])
          else:
            node.root=Node(randint(-10,10))
        else:
          #intermediate depth
          if random()>0.5:
            if random()<0.2:
             node.root=Node(TERMINALS[randint(0,len(TERMINALS)-1)])
            else:
             node.root=Node(randint(-10,10))
          else:
            func=FUNCTIONS[randint(0, len(FUNCTIONS)-1)]
            node.root = Node(func)
            if func in ['sin', 'cos']:
              node.root.left = cls().random_tree(grow, max_depth, depth=depth + 1).root
            else:
              node.root.left = cls().random_tree(grow, max_depth, depth=depth + 1).root
              node.root.right = cls().random_tree(grow, max_depth, depth=depth + 1).root



        return node

    def mutation(self):
     if random()<PROB_MUTATION:
      #mutate
      subtree=GPTree.random_tree(grow = True, max_depth =randint(2,10))
      if random()>0.5:
        self.root.right=subtree.root
      else:
        self.root.left=subtree.root



    def random_node(self):
     #select a random node from the tree
     nodes=self.get_nodes()
     if len(nodes) > 1:
      selected_node=choice(nodes[1:])
     else:
      selected_node=nodes[0] if nodes else None
     return selected_node

    def get_nodes(self):
      #Return a list of all nontermianl nodes in the tree
      nodes=[]
      self._get_nodes(self.root,nodes)
      return nodes

    def _get_nodes(self,node,nodes):
      #recursive function to traverse the tree an collect nodes
      if node.right and node.left:
        nodes.append(node)

      if node.left:
        self._get_nodes(node.left,nodes)
      if node.right:
        self._get_nodes(node.right,nodes)


    def crossover(self,other):
      #select a random node from each parent
      node1=self.random_node()
      node2=other.random_node()

      #swap the subtrees rooted at the selected nodes
      if node1 is not None and node2 is not None:
       temp=node1.left
       node1.left=node2.left
       node2.left=temp

      return self,other


def init_population():
  generated_population=[]
  for md in range(3,MAX_DEPTH+1):#create trees with depth in range 3 to 5
    for i in range(int(POP_SIZE/6)):
      generated_population.append(GPTree.random_tree(grow = True, max_depth = md))

    for i in range(int(POP_SIZE/6)):
      generated_population.append(GPTree.random_tree(grow = False, max_depth = md))

  return generated_population


def fitness(tree,dataset):
  return 1/(1+mean([abs(tree.compute({'x':ds[0]})-ds[1]) for ds in dataset]))



def Roulette_Wheel_Selection(population, fitnesses):
    # each tree has its
    # the probability of selecting an individual is directly proportional to its fitness value
    new_population = []
    sum_of_fitness_values = sum(fitnesses)
    probabilities = [fitness / sum_of_fitness_values for fitness in fitnesses]

    for _ in range(len(population)):#We want to select trees until we have the same population size as the initial one
        sum_upto_now = 0
        random_number = random()
        for j in range(len(population)):
            sum_upto_now += probabilities[j]
            if random_number <= sum_upto_now:
                # select jth tree
                new_population.append(population[j])
                break

    return new_population









def main():
  seed()
  dataset=generate_dataset(target_func)
  population=init_population()
  fitnesses = [fitness(population[i], dataset) for i in range(POP_SIZE)]
  best_of_run = None
  best_of_run_f = 0
  best_of_run_gen = 0

  for gen in range(GENERATIONS):
    population=Roulette_Wheel_Selection(population,fitnesses)
    for i in range(0,len(population),2):
      population[i],population[i+1]=population[i].crossover(population[i+1])

    #With a probability of 0.05, which is quite small, mutate each tree
    for tree in population:
      if random()<0.05:
        tree.mutation()

    #now compute the fitness function for new generation
    fitnesses = [fitness(population[i], dataset) for i in range(POP_SIZE)]

    if max(fitnesses)>best_of_run_f:
      best_of_run_f=max(fitnesses)
      best_of_run_gen=gen
      best_of_run=deepcopy(population[fitnesses.index(max(fitnesses))])
      print("____________________________")
      print(f"gen: {gen}    best_of_run_f: {round(max(fitnesses),3)}")
      print('best of run:')
      print(best_of_run)
    if best_of_run_f==1:break
  print("\n\n_________________________________________________\nEND OF RUN\nbest_of_run attained at gen " + str(best_of_run_gen) +\
          " and has f=" + str(round(best_of_run_f,3)))
  print(best_of_run)









if __name__=='__main__':
  main()






____________________________
gen: 0    best_of_run_f: 0.232
best of run:

             _add___
            /       \
      ____sin       div___
     /             /      \
  _mul            8       cos
 /    \                  /
-6     -3               x


____________________________
gen: 1    best_of_run_f: 0.315
best of run:

          _sub____
         /        \
      _cos       _div___
     /          /       \
  _cos         -6       cos
 /                     /
-3                    x


____________________________
gen: 4    best_of_run_f: 0.397
best of run:

       _____________________mul____________________
      /                                            \
   _mul___                                _________sin
  /       \                              /
-10       mul_________                _div___
         /            \              /       \
        8          ___sub         _cos       div
                  /      \       /          /   \
                div       8    

In [21]:
def target_func(x): # evolution's target
    return 3*x*x+3-math_cos(x)

def main():
  seed()
  dataset=generate_dataset(target_func)
  population=init_population()
  fitnesses = [fitness(population[i], dataset) for i in range(POP_SIZE)]
  best_of_run = None
  best_of_run_f = 0
  best_of_run_gen = 0

  for gen in range(GENERATIONS):
    population=Roulette_Wheel_Selection(population,fitnesses)
    for i in range(0,len(population),2):
      population[i],population[i+1]=population[i].crossover(population[i+1])

    #With a probability of 0.05, which is quite small, mutate each tree
    for tree in population:
      if random()<0.05:
        tree.mutation()

    #now compute the fitness function for new generation
    fitnesses = [fitness(population[i], dataset) for i in range(POP_SIZE)]

    if max(fitnesses)>best_of_run_f:
      best_of_run_f=max(fitnesses)
      best_of_run_gen=gen
      best_of_run=deepcopy(population[fitnesses.index(max(fitnesses))])
      print("____________________________")
      print(f"gen: {gen}    best_of_run_f: {round(max(fitnesses),3)}")
      print('best of run:')
      print(best_of_run)
    if best_of_run_f==1:break
  print("\n\n_________________________________________________\nEND OF RUN\nbest_of_run attained at gen " + str(best_of_run_gen) +\
          " and has f=" + str(round(best_of_run_f,3)))
  print(best_of_run)









if __name__=='__main__':
  main()

____________________________
gen: 0    best_of_run_f: 0.451
best of run:

                                                 _mul________________________________
                                                /                                    \
                         _____________________cos                                   _add_______________________
                        /                                                          /                           \
             _________add__________                                      ________cos                ___________cos
            /                      \                                    /                          /
     _____add___                ___mul____                         ___mul____                 ___mul____
    /           \              /          \                       /          \               /          \
  div_          mul         _div         _cos               ____mul         _sin           sub       

# 3-

In [27]:
from math import tan
def target_func(x): # evolution's target
    return tan(radians(x))*x*(-2)+3-math_cos(x)

def main():
  seed()
  dataset=generate_dataset(target_func)
  population=init_population()
  fitnesses = [fitness(population[i], dataset) for i in range(POP_SIZE)]
  best_of_run = None
  best_of_run_f = 0
  best_of_run_gen = 0

  for gen in range(GENERATIONS):
    population=Roulette_Wheel_Selection(population,fitnesses)
    for i in range(0,len(population),2):
      population[i],population[i+1]=population[i].crossover(population[i+1])

    #With a probability of 0.05, which is quite small, mutate each tree
    for tree in population:
      if random()<0.05:
        tree.mutation()

    #now compute the fitness function for new generation
    fitnesses = [fitness(population[i], dataset) for i in range(POP_SIZE)]

    if max(fitnesses)>best_of_run_f:
      best_of_run_f=max(fitnesses)
      best_of_run_gen=gen
      best_of_run=deepcopy(population[fitnesses.index(max(fitnesses))])
      print("____________________________")
      print(f"gen: {gen}    best_of_run_f: {round(max(fitnesses),3)}")
      print('best of run:')
      print(best_of_run)
    if best_of_run_f==1:break
  print("\n\n_________________________________________________\nEND OF RUN\nbest_of_run attained at gen " + str(best_of_run_gen) +\
          " and has f=" + str(round(best_of_run_f,3)))
  print(best_of_run)









if __name__=='__main__':
  main()

____________________________
gen: 0    best_of_run_f: 0.555
best of run:

          __________div____
         /                 \
      _add____            _mul____
     /        \          /        \
  _sin       _div       -6       _cos
 /          /    \              /
-3         -8     1            -6


____________________________
gen: 1    best_of_run_f: 0.677
best of run:

         ______________mul____
        /                     \
     _sub__________          _add
    /              \        /    \
  sin           ___sin     10     2
 /             /
7           _div
           /    \
          -2     2


____________________________
gen: 3    best_of_run_f: 0.87
best of run:

            ___sub____
           /          \
      ___div         _sub
     /      \       /    \
  _add       3     -1     4
 /    \
-9     0




_________________________________________________
END OF RUN
best_of_run attained at gen 3 and has f=0.87

            ___sub____
           /          \

# 4-

In [28]:

def target_func(x):
   # evolution's target
   if x<-1:
    return 2*x
   if x>-1 and x<5:
    return 4*x-28+x*x
   else:
    return 9




    return tan(radians(x))*x*(-2)+3-math_cos(x)

def main():
  seed()
  dataset=generate_dataset(target_func)
  population=init_population()
  fitnesses = [fitness(population[i], dataset) for i in range(POP_SIZE)]
  best_of_run = None
  best_of_run_f = 0
  best_of_run_gen = 0

  for gen in range(GENERATIONS):
    population=Roulette_Wheel_Selection(population,fitnesses)
    for i in range(0,len(population),2):
      population[i],population[i+1]=population[i].crossover(population[i+1])

    #With a probability of 0.05, which is quite small, mutate each tree
    for tree in population:
      if random()<0.05:
        tree.mutation()

    #now compute the fitness function for new generation
    fitnesses = [fitness(population[i], dataset) for i in range(POP_SIZE)]

    if max(fitnesses)>best_of_run_f:
      best_of_run_f=max(fitnesses)
      best_of_run_gen=gen
      best_of_run=deepcopy(population[fitnesses.index(max(fitnesses))])
      print("____________________________")
      print(f"gen: {gen}    best_of_run_f: {round(max(fitnesses),3)}")
      print('best of run:')
      print(best_of_run)
    if best_of_run_f==1:break
  print("\n\n_________________________________________________\nEND OF RUN\nbest_of_run attained at gen " + str(best_of_run_gen) +\
          " and has f=" + str(round(best_of_run_f,3)))
  print(best_of_run)









if __name__=='__main__':
  main()

____________________________
gen: 0    best_of_run_f: 0.047
best of run:

          ___add___
         /         \
      _sub         sin
     /    \       /
  _sin     7     2
 /
-8


____________________________
gen: 2    best_of_run_f: 0.085
best of run:

       ____mul_______
      /              \
   _add             _cos
  /    \           /
-10     -7       sin
                /
               x


____________________________
gen: 8    best_of_run_f: 0.208
best of run:

                  ___add___
                 /         \
      _________sub         sin
     /            \       /
  _mul___          7     2
 /       \
-3       add
        /   \
       x     7


____________________________
gen: 177    best_of_run_f: 0.348
best of run:

     _____________________add____
    /                            \
  sub________                   _div____
 /           \                 /        \
x           _sub___           -6       _cos
           /       \                  /
        

# 5-

In [32]:

def target_func(x,y): # evolution's target
    return 2*y+5*x*y-7*y*y

def generate_dataset(target_func): # generate 101 data points from target_func
    dataset = []
    for x in range(-100,101,2):
        y=random()
        x /= 100
        dataset.append([x,y, target_func(x,y)])
    return dataset

TERMINALS = ['x','y']

def fitness(tree,dataset):
  return 1/(1+mean([abs(tree.compute({'x':ds[0],'y':ds[1]})-ds[2]) for ds in dataset]))

def main():
  seed()
  dataset=generate_dataset(target_func)
  population=init_population()
  fitnesses = [fitness(population[i], dataset) for i in range(POP_SIZE)]
  best_of_run = None
  best_of_run_f = 0
  best_of_run_gen = 0

  for gen in range(GENERATIONS):
    population=Roulette_Wheel_Selection(population,fitnesses)
    for i in range(0,len(population),2):
      population[i],population[i+1]=population[i].crossover(population[i+1])

    #With a probability of 0.05, which is quite small, mutate each tree
    for tree in population:
      if random()<0.05:
        tree.mutation()

    #now compute the fitness function for new generation
    fitnesses = [fitness(population[i], dataset) for i in range(POP_SIZE)]

    if max(fitnesses)>best_of_run_f:
      best_of_run_f=max(fitnesses)
      best_of_run_gen=gen
      best_of_run=deepcopy(population[fitnesses.index(max(fitnesses))])
      print("____________________________")
      print(f"gen: {gen}    best_of_run_f: {round(max(fitnesses),3)}")
      print('best of run:')
      print(best_of_run)
    if best_of_run_f==1:break
  print("\n\n_________________________________________________\nEND OF RUN\nbest_of_run attained at gen " + str(best_of_run_gen) +\
          " and has f=" + str(round(best_of_run_f,3)))
  print(best_of_run)









if __name__=='__main__':
  main()

____________________________
gen: 0    best_of_run_f: 0.39
best of run:

                                     _mul____________________________
                                    /                                \
                    ______________sin                     ___________cos
                   /                                     /
       __________mul__________                        _add_______
      /                       \                      /           \
   _mul____                ___cos               ___sin          _sin
  /        \              /                    /               /
-10       _mul         _mul                 _mul             cos
         /    \       /    \               /    \           /
        -3     x     -6     9             -7     y         x


____________________________
gen: 16    best_of_run_f: 0.446
best of run:

                       _______mul___
                      /             \
     _______________sub___          add________